In [19]:
import pandas as pd 
import numpy as np 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
print('done')

done


In [20]:
#to get the df
w = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
wt = w[0]
tw = wt[wt["Borough"] != "Not assigned"].reset_index(drop=True)
tw

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [21]:
# get the shape of our df
tw.shape

(103, 3)

In [22]:
#to add a Latitude & Longitude into the df
lalo = pd.read_csv('http://cocl.us/Geospatial_data')
ll = pd.merge(tw,lalo,on="Postal Code")
ll.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [23]:
tor = ll[ll['Borough'].astype(str).str.contains("Toronto")].reset_index(drop=True)

In [31]:
cl =tor.drop(columns=["Postal Code","Borough", "Neighbourhood"])

# use kMeans
k = KMeans(n_clusters=5).fit(cl)

# make a new column named labels contained kMeans labels
tor["labels"] = k.labels_

# make the new column labels to be shown at the beginning of the dataframe
fixed_columns = [tor.columns[-1]] + list(tor.columns[:-1])
tor_n = tor[fixed_columns]

In [38]:
map_cl = folium.Map(location=[43.654260, -79.360636], zoom_start=11)

x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
color = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in color]

markers_colors = []
for lat, lon, poi, cluster in zip(tor_n['Latitude'],
                                  tor_n['Longitude'],
                                  tor_n['Neighbourhood']
                                  , tor_n['labels']):
    
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cl)
       
map_cl

AttributeError: 'numpy.ndarray' object has no attribute 'rgb2hex'